설정이 필요한 부분)
1. service = Service('/Users/myo/development/chromedriver')  # chromedriver 경로를 지정해주세요 -> 컴퓨터의 크롬드라이브 경로를 찾아서 설정해주기. 아마 크롬드라이버를 다운 받아야 할듯.
2. input_file = '/Users/myo/Desktop/Kangs/interv_sr_list.csv' 부분과 output_file = '/Users/myo/Desktop/Kangs/picos_good.csv' 부분의 파일들 경로 지정해두기. 참고로 이 ipynb 파일은 맥북 기준이고, 윈도우는 맥북과 다르게 경로 설정이 \ 역슬래시임. interv_sr_list는 논문 제목을 가져올 csv 파일이고 얘는 github에 올라와 있음, picos_good은 추출한 피코 정보를 기입할 새로운 파일이름임. Picos_good.csv 파일 이름은 마음대로 지정해도 됨
3. 처리할 논문 범위 설정
    start_index = n - 1  # 데이터프레임 인덱스는 0부터 시작
    end_index = m -> 컴퓨터를 나눠서 돌릴 것이기 때문에 n, m 숫자 지정(원하는 논문이 예를 들어 2001번부터 2200까지면 n=2001, m=2200으로 n, m만 바꾸면 되게 코딩되어 있음)
4. 오류가 날 수 있는 포인트 : chrome driver 및 파일들 경로 지정 오류, chrome과 Chromedriver 버전 안맞는 오류.
5. 혹시라도 오류 생기면 GPT한테 물어보기

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import os

def get_first_paper_link(driver, title):
    # Cochrane Library 검색 페이지로 이동
    search_url = "https://www.cochranelibrary.com/search"
    driver.get(search_url)

    # 검색창 찾기 및 논문 제목 입력
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "searchText"))
    )
    search_box.clear()
    search_box.send_keys(title)
    search_box.send_keys("\n")

    # 첫 번째 검색 결과의 링크 추출
    first_result = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".search-results-item .result-title a"))
    )
    paper_link = first_result.get_attribute('href')
    
    return paper_link

def extract_pico(url):
    # Selenium 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # GUI 없이 실행
    service = Service('/Users/myo/development/chromedriver')  # chromedriver 경로를 지정해주세요
    driver = webdriver.Chrome(service=service, options=chrome_options)

    driver.get(url)
    
    # PICO 섹션이 로드될 때까지 대기
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.ID, "pico")))
    
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 스크롤 후 콘텐츠 로드를 위한 대기

    # 페이지 소스 가져오기
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    pico = {}
    pico_section = soup.find('section', class_='pico-section', id='pico')
    
    if pico_section:
        for column in pico_section.find_all('div', class_='pico-column'):
            pico_type = column['class'][1]  # Population, Intervention, Comparison, Outcome
            terms = [a.text.strip() for a in column.find_all('a')]
            pico[pico_type] = terms

    driver.quit()
    return pico

def process_papers(start_index, end_index, titles_df, output_file):
    # Selenium 설정
    chrome_options = Options()
    service = Service('/Users/myo/development/chromedriver')  # chromedriver 경로를 지정해주세요
    driver = webdriver.Chrome(service=service, options=chrome_options)

    pico_data = []

    # 지정된 범위의 논문에 대해 링크 추출 및 PICO 정보 추출
    for index in range(start_index, end_index):
        if index >= len(titles_df):
            break
        
        row = titles_df.iloc[index]
        title = row['Title']
        year_issue = f"{row['Year']}-{row['Issue']}"
        print(f"Processing {index+1}/{len(titles_df)}: {title}")

        try:
            # 논문의 첫 번째 링크 추출
            paper_link = get_first_paper_link(driver, title)
            print(f"Found paper link: {paper_link}")

            # PICO 정보 추출
            pico_info = extract_pico(paper_link)
            pico_info["No."] = index + 1
            pico_info["Year-Issue"] = year_issue
            pico_info["Title"] = title  # PICO 정보에 제목 추가
            pico_data.append(pico_info)
            print(f"PICO data for {title}: {pico_info}")

        except Exception as e:
            print(f"Error processing title: {title}, error: {e}")
            # 오류 발생 시, 제목만 저장하고 나머지는 공백으로 처리
            pico_data.append({"No.": index + 1, "Year-Issue": year_issue, "Title": title, "Population": "", "Intervention": "", "Comparison": "", "Outcome": ""})

    driver.quit()

    # 결과를 DataFrame으로 변환
    df = pd.DataFrame(pico_data, columns=["No.", "Year-Issue", "Title", "Population", "Intervention", "Comparison", "Outcome"])

    # 기존 CSV 파일에 추가
    if os.path.exists(output_file):
        # 파일의 끝에 데이터를 추가
        with open(output_file, 'a') as f:
            df.to_csv(f, header=False, index=False)
    else:
        # 파일이 없으면 새로 작성
        df.to_csv(output_file, index=False)

    print(f"Saved PICO data to {output_file}")

def main():
    # CSV 파일에서 논문 제목 읽기
    input_file = '/Users/myo/Desktop/Kangs/interv_sr_list.csv'
    titles_df = pd.read_csv(input_file)

    # 출력 파일 설정
    output_file = '/Users/myo/Desktop/Kangs/picos_good.csv'

    # 특정 논문 범위를 설정
    n = int(input("Start index (n): "))  # 시작 논문 번호
    m = int(input("End index (m): "))    # 종료 논문 번호

    if n < 1 or m > len(titles_df) or n > m:
        print("Invalid range. Please check the indices.")
        return

    # 처리할 논문 범위 설정
    start_index = n - 1  # 데이터프레임 인덱스는 0부터 시작
    end_index = m

    # 논문 처리
    process_papers(start_index, end_index, titles_df, output_file)

if __name__ == "__main__":
    main()
